In [ ]:
%matplotlib inline

import json
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
df_sf = pd.read_csv('mortgage_vs_rent_sf.csv', dtype={'postal_code': str})
df_mi = pd.read_csv('mortgage_vs_rent_mi.csv', dtype={'postal_code': str})

In [ ]:
df_sf = df_sf.groupby(['year', 'postal_code']).mean()[['morgate_vs_rent']].reset_index()
df_mi = df_mi.groupby(['year', 'postal_code']).mean()[['morgate_vs_rent']].reset_index()

In [ ]:
mortgage_zips = df_sf['postal_code'].unique().tolist()+df_mi['postal_code'].unique().tolist()
mortgage_zips

In [ ]:
with open('../../data/geojson/ca_california_zip_codes_geo.min.json') as response:
    zipcodes_sf = json.load(response)
    
with open('../../data/geojson/fl_florida_zip_codes_geo.min.json') as response:
    zipcodes_mi = json.load(response)
# ca_california_zip_codes_geo.min
# fl_florida_zip_codes_geo.min
# ny_new_york_zip_codes_geo.min
# tx_texas_zip_codes_geo.min

In [ ]:
json_zips_sf = []
for z in zipcodes_sf['features']:
    json_zips_sf.append(z['properties']['ZCTA5CE10'])

json_zips_mi = []
for z in zipcodes_mi['features']:
    json_zips_mi.append(z['properties']['ZCTA5CE10'])

In [ ]:
json_zips_df_sf = {'postal_code': [], 'postal_code_full': []}
for z in json_zips_sf:
    temp = z[:3]+'00'
    if temp in mortgage_zips:
        json_zips_df_sf['postal_code_full'].append(z)
        json_zips_df_sf['postal_code'].append(temp)
json_zips_df_sf = pd.DataFrame(json_zips_df_sf)
json_zips_df_sf

In [ ]:
json_zips_df_mi = {'postal_code': [], 'postal_code_full': []}
for z in json_zips_mi:
    temp = z[:3]+'00'
    if temp in mortgage_zips:
        json_zips_df_mi['postal_code_full'].append(z)
        json_zips_df_mi['postal_code'].append(temp)
json_zips_df_mi = pd.DataFrame(json_zips_df_mi)
json_zips_df_mi

In [ ]:
df_sf = pd.merge(df_sf, json_zips_df_sf, how='left', on='postal_code')
df_mi = pd.merge(df_mi, json_zips_df_mi, how='left', on='postal_code')

In [ ]:
df_sf_20 = df_sf[df_sf['year']==2020][['morgate_vs_rent', 'postal_code_full', 'postal_code']]
df_mi_20 = df_mi[df_mi['year']==2020][['morgate_vs_rent', 'postal_code_full', 'postal_code']]

In [ ]:
df_sf_20.sample(20)

In [ ]:
# SF
# 37.775540, -122.442670

# MI
# 25.933817, -80.183161

In [ ]:
fig = px.choropleth_mapbox(df_sf_20, geojson=zipcodes_sf, locations='postal_code_full', color='morgate_vs_rent',
                           featureidkey='properties.ZCTA5CE10',
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           zoom=10, center = {"lat": 37.775540, "lon": -122.442670},
                           opacity=0.5,
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth_mapbox(df_mi_20, geojson=zipcodes_mi, locations='postal_code_full', color='morgate_vs_rent',
                           featureidkey='properties.ZCTA5CE10',
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           zoom=10, center = {"lat": 25.933817, "lon": -80.183161},
                           opacity=0.5,
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()